In [1]:
import autograd.numpy as np
import pyipopt
from autograd import grad, hessian, jacobian

In [2]:
import matplotlib.pyplot as plt
from counterfactual_functions import *
from helper import *

username='boraozaltun'

data_dict = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/final_data/data_subset_20200304.pickle')

reg_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/reg_2_num.pickle')
comm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/comm_2_num.pickle')

n = data_dict['n']
g = data_dict['g']
k = data_dict['k']

data_dict['R_hat'] = np.ones((n, g))
data_dict['R_hat'][reg_2_num['usa'], 3] = 2.
data_dict['R_hat'][reg_2_num['usa'], 4] = 2.
X_0 = np.ones(n*g + n)*1.0

#eval_g = lambda x: reduced_counterfactual(x, data_dict) # System of residuals
eval_F = lambda x: reduced_counterfactual_overdetermined(x, data_dict)
eval_f = lambda x: np.sum(eval_F(x)**2)

eval_grad_f = grad(eval_f)



In [3]:
# np.linspace(0, len(X_0) -1, len(X_0)).astype(int)

In [4]:
nvar = X_0.shape[0]
x_L = np.zeros((nvar), dtype=float)
x_U = np.ones((nvar), dtype=float) * 100.0

ncon = 0


g_L = np.array([], dtype=float)
g_U = np.array([], dtype=float)



In [5]:
def eval_g(X, user_data=None):
    """
    Evaluate the constraint functions.
    """
    return np.array([], dtype=float)

nnzj = 0

def eval_jac_g(X, flag, user_data=None):
    """
    Evaluate the sparse Jacobian of constraint functions g.
    @param X: parameter values
    @param flag: this asks for the sparsity structure
    """
    print('eval_jac_g')
    print(X)
    print(flag)
    print(user_data)
    print()
    #XXX
    if flag:
        rows = np.array([], dtype=int)
        cols = np.array([], dtype=int)
        return (rows, cols)
    else:
        return np.array([], dtype=float)

nnzh = int((nvar*(nvar+1))/2)

def apply_new(x):
    return True

In [6]:
eval_f(X_0)

0.49689295554623275

In [7]:

nlp = pyipopt.create(
            nvar,
            x_L,
            x_U,
            ncon,
            g_L,
            g_U,
            nnzj,
            nnzh,
            eval_f,
            eval_grad_f,
            eval_g,
            eval_jac_g)

#nlp.num_option('tol', 1e-5)
nlp.int_option('max_iter', 20000)



True

In [8]:
results = nlp.solve(X_0)

eval_jac_g
[0.000e+000 1.136e-321 0.000e+000 1.136e-321 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e+000
 0.000e+000 0.000e+000 0.000e+000 0.000e+000 0.000e

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/autograd/numpy/numpy_vjps.py:444: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return lambda g: g[idxs]
/Users/boraozaltun/GitHub/fossilfuelnetworkscience/General_Equilibrium_Model/solver_ipopt/counterfactual_functions.py:130: RuntimeWarning: invalid value encountered in power
  part1 = data['phi_R'] * (r_hat ** (1- data['rho_g'].reshape((1, data['g']))))
/Users/boraozaltun/GitHub/fossilfuelnetworkscience/General_Equilibrium_Model/solver_ipopt/counterfactual_functions.py:131: RuntimeWarning: invalid value encountered in power
  part2 = data['phi_L_g'] * (w_hat.reshape((data['n'], 1)) ** (1 - data['rho_g'].reshape((1, data['g']))) )
/Users/boraozaltun/GitHub/fossilfuelnetworkscience/Ge

In [9]:
x, zl, zu, constraint_multipliers, obj, status = results

In [10]:
nlp.close()

True

In [11]:
## If this is still not the right result, I think I might be doing something wrong in setting IPOPT up.

## Because this makes no sense. i am giving it a point where it is the minimum, but somehow it is still not
## converginng? Try with scipy and see what happens!

In [12]:
x

array([0.99638244, 1.0031111 , 1.00233304, 0.90787777, 0.92446198,
       0.99695323, 0.99370007, 1.00042783, 1.002508  , 0.91174312,
       0.99815227, 0.98947063, 0.99686811, 0.9999242 , 1.00174597,
       0.91310255, 0.85180158, 0.99621187, 1.00273   , 1.00462621,
       1.00205425, 0.89398586, 0.95554369, 1.00671484, 0.99446063,
       1.00490309, 0.99905143, 0.966815  , 0.90370527, 0.97619842,
       1.00850625, 1.00859521, 1.00680665, 0.88409385, 0.89992906,
       1.00737291, 0.99853613, 1.00098306, 1.00210141, 0.91387909,
       0.93801477, 0.99356972, 0.97035618, 0.98990307, 0.99640525,
       0.92073314, 0.87491501, 0.99646766, 0.99774589, 0.9982531 ,
       1.00495985, 0.8674229 , 0.93200691, 0.99086301, 0.98472281,
       0.99527805, 0.98999784, 0.90248605, 0.9644574 , 0.99408297,
       0.96651766, 0.99500456, 0.99916302, 0.89633968, 0.87738504,
       0.96360313, 0.99722202, 1.00050068, 1.00073535, 0.91397543,
       0.93189438, 0.99728072, 0.98665152, 0.9826031 , 0.99581

In [13]:
obj

0.00025343158727632596

In [14]:
status

1

In [15]:
r_hat_star, w_hat_star = get_values_from_X_reduced(x, data_dict)

In [16]:
E_hat_star = get_E_hat(w_hat_star, r_hat_star, data_dict)

In [17]:
E_hat_star.sum()

132.9725249351637

In [18]:
get_E_hat(np.ones((n,1)), np.ones((n, g)), data_dict).sum()

134.0038482256796

In [19]:
print(r_hat_star[reg_2_num['usa'], 3], r_hat_star[reg_2_num['usa'], 4])

0.6367978488351901 0.5295005949710531


In [20]:
print(r_hat_star[reg_2_num['usa'],:])

[1.01572919 1.01005433 1.01281787 0.63679785 0.52950059 1.00785293]


In [21]:
print(w_hat_star[reg_2_num['usa'],0])

0.9999980841666524
